# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [16]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
# Import API key
from api_keys_laura import geoapify_key

In [17]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities_laura.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Pressure,Cloudiness,Wind Speed,Country,Date
0,0,broome,42.2506,-75.8330,60.67,68.0,1020.0,95.0,0.94,0.94,1.717298e+09
1,1,dryden,49.7833,-92.7503,59.29,51.0,1009.0,0.0,5.75,5.75,1.717298e+09
2,2,atafu village,-8.5421,-172.5159,84.04,73.0,1009.0,20.0,5.10,5.10,1.717298e+09
3,3,sepuka,-4.7500,34.5333,60.08,81.0,1016.0,99.0,6.02,6.02,1.717298e+09
4,4,lompoc,34.6391,-120.4579,58.78,84.0,1014.0,0.0,13.80,13.80,1.717298e+09


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [18]:
# Create a DataFrame
data = {
    'City_ID': [0, 1, 2, 3, 4],
    'City': ['Broome', 'Dryden', 'Atafu Village', 'Sepuka', 'Lompoc'],
    'Lat': [42.2506, 49.7833, -8.5421, -4.7500, 34.6391],
    'Lng': [-75.8330, -92.7503, -172.5159, 34.5333, -120.4579],
    'Humidity': [68.0, 51.0, 73.0, 81.0, 84.0]
}

city_data_df = pd.DataFrame(data)

# Function to plot the map
def plot_city_map(city_data_df):
    city_plot = city_data_df.hvplot.points(
        'Lng',
        'Lat',
        size='Humidity',  # Size of the points based on humidity
        hover_cols=['City', 'Humidity'],
        geo=True,
        tiles='CartoLight',
        title='City Humidity Map'
    )
    return city_plot

# Plot the map
plot = plot_city_map(city_data_df)
plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Define ideal humidity condition
ideal_humidity_condition = city_data_df['Humidity'] < 70

# Filter the DataFrame
ideal_cities_df = city_data_df[ideal_humidity_condition]

# Display the filtered DataFrame
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Humidity
0,0,Broome,42.2506,-75.8330,68.0
1,1,Dryden,49.7833,-92.7503,51.0


In [20]:
print(city_data_df.columns)

Index(['City_ID', 'City', 'Lat', 'Lng', 'Humidity'], dtype='object')


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Create a new DataFrame called hotel_df
hotel_df = city_data_df[['City', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Lat,Lng,Humidity,Hotel Name
0,Broome,42.2506,-75.8330,68.0,
1,Dryden,49.7833,-92.7503,51.0,
2,Atafu Village,-8.5421,-172.5159,73.0,
3,Sepuka,-4.7500,34.5333,81.0,
4,Lompoc,34.6391,-120.4579,84.0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set the geographical coordinates
data = {
    'City_ID': [0, 1, 2, 3, 4],
    'City': ['Broome', 'Dryden', 'Atafu Village', 'Sepuka', 'Lompoc'],
    'Lat': [42.2506, 49.7833, -8.5421, -4.7500, 34.6391],
    'Lng': [-75.8330, -92.7503, -172.5159, 34.5333, -120.4579],
    'Humidity': [68.0, 51.0, 73.0, 81.0, 84.0]
}
# Set the parameters for the type of search
limit = 20
radius = 1000
filters = f"circle:{lng},{lat},{radius}"
bias = f"proximity:{lng},{lat}"

# set up a parameters dictionary
params = {
    "filter":filters,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

In [23]:
# Set parameters to search for a hotel
radius = 10000  # Define the radius in meters
params = {
    'type': 'hotel',  # Specify the type of place to search for
    'radius': radius,  # Set the search radius
    'limit': 1,  # Limit the number of results to 1
    'api_keys_laura': geoapify_key   # Use your Geoapify API key here
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"distance(point({lng},{lat}), point.lonlat) < {radius}"
    params["bias"] = f"proximity(point({lng},{lat}))"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Broome - nearest hotel: No hotel found
Dryden - nearest hotel: No hotel found
Atafu Village - nearest hotel: No hotel found
Sepuka - nearest hotel: No hotel found
Lompoc - nearest hotel: No hotel found


,City,Lat,Lng,Humidity,Hotel Name
0,Broome,42.2506,-75.8330,68.0,No hotel found
1,Dryden,49.7833,-92.7503,51.0,No hotel found
2,Atafu Village,-8.5421,-172.5159,73.0,No hotel found
3,Sepuka,-4.7500,34.5333,81.0,No hotel found
4,Lompoc,34.6391,-120.4579,84.0,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
# Configure the map plot
city_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    size='Humidity',
    hover_cols=['City', 'Humidity', 'Hotel Name', 'Country'],  # Add 'Hotel Name' and 'Country' to hover_cols
    geo=True,
    tiles='CartoLight',
    title='City Humidity Map with Hotel Information'
)

# Display the map
city_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

In [25]:
#Note= I tried numerous times to change the code and read my API documents but I could'nt manage to change the result when looking for the neartest hotels, many of my teammates had the same issue, I apologize. 